In [93]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
import sklearn
from sklearn import preprocessing
import sys

sys.path.insert(1, "./code/")
from Eval import Eval

print(tf.__version__)

1.12.0


In [68]:
traindata = pd.read_pickle('data/traindata.pickle')
valdata   = pd.read_pickle('data/valdata.pickle')
testdata  = pd.read_pickle('data/testdata.pickle')

def numerize(item):
    if item == 'MWS':
        return 0
    elif item == 'HPL':
        return 1
    elif item == 'EAP':
        return 2
    raise ValueError('Author not recognized:', item)
   
encoder = sklearn.preprocessing.LabelEncoder()
encoder.fit(traindata.author)


Y_train = tf.keras.utils.to_categorical(encoder.transform(traindata.author))#np.array([numerize(x) for x in list(traindata.author)])
Y_val = tf.keras.utils.to_categorical(encoder.transform(valdata.author))#np.array([numerize(x) for x in list(valdata.author)])
Y_test = tf.keras.utils.to_categorical(encoder.transform(testdata.author))#np.array([numerize(x) for x in list(testdata.author)])


In [76]:
encoder.classes_

array(['EAP', 'HPL', 'MWS'], dtype=object)

In [3]:
# textual feature engineering
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=STOPWORDS, max_features=100)

X_train = train_text_feats = vectorizer.fit_transform(traindata.text).todense()
X_val = val_text_feats = vectorizer.transform(valdata.text).todense() 
X_test = test_text_feats = vectorizer.transform(testdata.text).todense() 


In [4]:
# train_text_feats_df = pd.read_pickle('data/train_text_feats_df.pickle')
# val_text_feats_df   = pd.read_pickle('data/val_text_feats_df.pickle')
# test_text_feats_df  = pd.read_pickle('data/test_text_feats_df.pickle')

# for x in [train_text_feats_df, val_text_feats_df, test_text_feats_df]:
#     print(x.shape)

In [5]:

# Y_train_file = open('data/Y_train.pickle', 'wb')
# Y_val_file = open('data/Y_val.pickle', 'wb')
# Y_test_file = open('data/Y_test.pickle', 'wb')

# pickle.dump(Y_train, Y_train_file)
# pickle.dump(Y_val, Y_val_file)
# pickle.dump(Y_test, Y_test_file)

# for x in [Y_train_file, Y_val_file, Y_test_file]:
#     x.close()

In [7]:

# # Create a sigmoid layer:
# tf.keras.layers.Dense(64, activation='sigmoid')
# # Or:
# tf.keras.layers.Dense(64, activation=tf.sigmoid)

# # A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
# tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# # A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
# tf.keras.layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# # A linear layer with a kernel initialized to a random orthogonal matrix:
# tf.keras.layers.Dense(64, kernel_initializer='orthogonal')

# # A linear layer with a bias vector initialized to 2.0s:
# tf.keras.layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

In [69]:
model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
tf.keras.layers.Dense(100, activation='relu'),
# Add another:
tf.keras.layers.Dense(100, activation='relu'),
# Add a softmax layer with 10 output units:
tf.keras.layers.Dense(3, activation='softmax')])


In [70]:
# Configure a model for categorical classification.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy]
              )

In [42]:
type(Y_train)

numpy.ndarray

In [71]:

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))

Train on 15664 samples, validate on 1958 samples
Epoch 1/10
15664/15664 [==============================] - 2s 139us/step - loss: 1.0100 - categorical_accuracy: 0.4892 - val_loss: 0.9715 - val_categorical_accuracy: 0.5204
Epoch 2/10
15664/15664 [==============================] - 1s 88us/step - loss: 0.9638 - categorical_accuracy: 0.5280 - val_loss: 0.9841 - val_categorical_accuracy: 0.5266
Epoch 3/10
15664/15664 [==============================] - 1s 89us/step - loss: 0.9510 - categorical_accuracy: 0.5402 - val_loss: 0.9539 - val_categorical_accuracy: 0.5383
Epoch 4/10
15664/15664 [==============================] - 1s 85us/step - loss: 0.9442 - categorical_accuracy: 0.5469 - val_loss: 0.9486 - val_categorical_accuracy: 0.5414
Epoch 5/10
15664/15664 [==============================] - 1s 86us/step - loss: 0.9349 - categorical_accuracy: 0.5502 - val_loss: 0.9590 - val_categorical_accuracy: 0.5475
Epoch 6/10
15664/15664 [==============================] - 1s 88us/step - loss: 0.9279 - categor

In [53]:
[7,8,9].index

(15664, 100)

In [82]:
# returns loss, accuracy
result = model.predict(X_test, verbose=2 )

In [89]:
row = result[0]
mx = max(row)
list(row).index(mx)
row

array([0.49292427, 0.29339072, 0.21368499], dtype=float32)

In [90]:
label_lookup = {}
for idx, label in enumerate(encoder.classes_):
    label_lookup[idx] = label
label_lookup

def best_label(row, lookup):
    mx = max(row)
    return lookup[list(row).index(mx)]

preds = [best_label(x, label_lookup) for x in result]
preds
#[1.1141872412706433, 0.5380684721512519]

['EAP',
 'EAP',
 'HPL',
 'HPL',
 'MWS',
 'EAP',
 'HPL',
 'HPL',
 'MWS',
 'EAP',
 'HPL',
 'HPL',
 'HPL',
 'HPL',
 'HPL',
 'EAP',
 'HPL',
 'MWS',
 'EAP',
 'HPL',
 'EAP',
 'HPL',
 'MWS',
 'HPL',
 'EAP',
 'HPL',
 'MWS',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'HPL',
 'EAP',
 'EAP',
 'MWS',
 'HPL',
 'EAP',
 'EAP',
 'HPL',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'HPL',
 'HPL',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'HPL',
 'EAP',
 'EAP',
 'EAP',
 'HPL',
 'HPL',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'HPL',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'HPL',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'HPL',
 'EAP',
 'MWS',
 'HPL',
 'HPL',
 'MWS',
 'EAP',
 'HPL',
 'EAP',
 'EAP',
 'EAP',
 'HPL',
 'EAP',
 'HPL',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'EAP',


In [95]:
accuracy = Eval.get_accuracy(preds, list(testdata.author))
print("Accuracy: ", accuracy)

Accuracy:  0.5380684721512519


In [ ]:
import time
timenow = time.asctime( time.localtime(time.time()) )
print("Finished at: ", timenow)